In [1]:
import numpy as np
import pandas as pd
import datetime
import recommendations as rec
import dataset
import sys
from sklearn.model_selection import train_test_split

In [2]:
def round_of_rating(number):
    """
    Round a number to the closest half integer.
    """

    return round(number * 2) / 2

In [3]:
users_pd = pd.DataFrame.from_dict(data=dataset.openJson('datasets/prefs_filtered.json'),\
                                  orient='columns', dtype='float16')

In [4]:
users_pd.head(5)

,1,10,100,1000,100044,100068,100083,100106,100108,100159,...,99764,99787,998,99813,999,99910,99912,99917,99957,99996
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
print(round(sys.getsizeof(users_pd)/float(1024**2), 4), 'MB')

5370.6681 MB


In [6]:
users_pd.shape

(248781, 11287)

In [7]:
# xyz = users_pd.to_dict(orient='index')

#### Calculating label for stratified train-test split

In [8]:
# movies_prefs = dataset.openJson('datasets/prefs_filtered.json')
# users_prefs = dataset.transformPrefs(movies_prefs)

In [9]:
# # info concerning possible thresholds
# for i in [0.1, 0.25, 0.3, 0.4, 0.5, 0.6, 0.75, 0.8, 0.9]:
#     print('\n'+'percentile:', str(i))
#     print(len(list(rec.getPopularMovies(prefs=users_prefs, percentile=i))))

# # users_dict = getUsersDict(prefs_file='datasets/prefs.json', raw_file=False)
# # nr_rows = len(users_dict)

In [10]:
# # stratify by mean
# y = users_pd.mean(axis='columns', skipna=True, numeric_only=True).apply(round_of_rating)

In [11]:
# stratify by number of rated movies
users_pd['count'] = users_pd.count(axis='columns', numeric_only=True)

In [12]:
users_pd['label'] = np.select(
    [
        users_pd['count'].between(0, 16, inclusive=True),
        users_pd['count'].between(17, 35, inclusive=True),
        users_pd['count'].between(36, 102, inclusive=True),
        users_pd['count'].between(103, 1000000, inclusive=True)
    ],
    [
        'q1',
        'q2',
        'q3',
        'q4'
    ],
    default='unknown'
)

In [13]:
y = users_pd['label']

In [14]:
len(y)

248781

In [15]:
y.head(10)

1         q2
10        q1
1000      q1
10000     q1
100000    q3
100001    q2
100002    q4
100003    q3
100004    q4
100006    q2
Name: label, dtype: object

In [16]:
users_pd[['count', 'label']].head(10)

,count,label
1,27,q2
10,13,q1
1000,6,q1
10000,9,q1
100000,40,q3
100001,18,q2
100002,201,q4
100003,37,q3
100004,282,q4
100006,17,q2


In [17]:
users_pd.drop(labels=['count', 'label'], axis='columns', inplace=True)

In [18]:
# y.hist(bins=4)

In [32]:
y.groupby(by=y).count()

label
q1    65527
q2    59217
q3    61974
q4    62063
Name: label, dtype: int64

In [35]:
len(y)

248781

In [36]:
len(users_pd)

248781

### Performing train-test split

In [44]:
X_train, X_test, y_train, y_test = train_test_split(users_pd, y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y
                                                   )

In [45]:
y_train.groupby(y_train).count()

label
q1    52421
q2    47374
q3    49579
q4    49650
Name: label, dtype: int64

In [46]:
y_test.groupby(y_test).count()

label
q1    13106
q2    11843
q3    12395
q4    12413
Name: label, dtype: int64

#### dict to ndarray conversion

In [ ]:
# # utworzenie prostej arrajki o n-wymiarach
# test_array = np.zeros([nr_rows, nr_cols], float)

In [ ]:
# # i=0
# print(datetime.datetime.now())
# for movie_pos, popular_movie in enumerate(popular_movies):
# #     i+=1
#     # if i%100 == 0:
#     for user_pos, (user, ratings) in enumerate(users_dict.items()):
#         for movie, rating in ratings.items():
#             if popular_movie == movie:
#                 test_array.itemset((user_pos, movie_pos), rating)
# #                 print(datetime.datetime.now(), '----- user:', user, '--- movie:', popular_movie, '--- rating:', rating)
                
# test_array[test_array == 0] = np.NaN
# print(datetime.datetime.now())

In [ ]:
# np.save(file='datasets/users_filtered_ratings.npy')
# print(datetime.datetime.now())

In [ ]:
# print('max of whole array:', np.nanmax(test_array))
# print('max among users:', [x for x in np.nanmax(test_array, axis=0).tolist() if ~np.isnan(x)])
# print('min among users:', [x for x in np.nanmin(test_array, axis=0).tolist() if ~np.isnan(x)])